In [1]:
import pandas as pd
import numpy as np
import pymongo
from bs4 import BeautifulSoup as bs
import pprint
#import matplotlib.pyplot as plt

In [2]:
#conn = "mongodb://localhost:27017"
client = pymongo.MongoClient()

In [3]:
db = client.realtyaus
collection = db.main_pages

In [4]:
df = pd.DataFrame(list(collection.find()))

In [19]:
def convert_data(data):
    '''
    Convert html data from realtyaus database into dataframe by obtaining key elements from 
    each webpage.
    
    '''
    
    #initiate beautiful soup html.parser
    soup = bs(data, 'html.parser')
    
    #utilized find_all to obtain elements location
    listings = soup.find_all(class_='articleset listings colset_4')
    mlsids = listings[0].find_all('li', class_="data-mlsid")
    subdivs = [x.get_text(strip=True) for x in listings[0].find_all('li', class_="data-subdivision")]
    address = [x.get_text() for x in listings[0].find_all('li', class_="data-address")]
    locs = listings[0].find_all('li', class_="data-location")
    beds = listings[0].find_all('span', class_="details-bedrooms")
    baths = listings[0].find_all('span', class_="details-bathrooms")
    sqfts = listings[0].find_all('span', class_="details-sqft")
    acres = listings[0].find_all('span', class_="details-acres")
    dtypes = listings[0].find_all('li', class_="data-type")
    prices = listings[0].find_all('li', class_="data-price")
    status = listings[0].find_all('li', class_="data-status")
    statusexts = listings[0].find_all('li', class_="data-statusextra")
    photos = listings[0].find_all(class_="photo")
    
    #list comprehension 
    mlsid = [x.get_text(strip=True) for x in mlsids]
    loc = [x.get_text(strip=True) for x in locs]
    bed = [x.get_text(strip=True) for x in beds]
    bath = [x.get_text(strip=True) for x in baths]
    sqft = [x.get_text(strip=True) for x in sqfts]
    acre = [x.get_text(strip=True) for x in acres]
    dtype = [x.get_text(strip=True) for x in dtypes]
    price = [x.get_text(strip=True) for x in prices]
    statu = [x.get_text(strip=True) for x in status]
    statusext = [x.get_text(strip=True) for x in statusexts]
    photo = [x.img['data-src'] for x in photos]
    
    #create a dictionary 
    data = {
        'mlsid': mlsid,
        'subdiv': subdiv,
        'location': loc,
        'bedrooms': bed,
        'bathrooms': bath, 
        'sqft': sqft,
        'acre': acre,
        'data_type': dtype,
        'price': price,
        'status': statu,
        'status_extra': statusext,
        'photo': photo
    }
    
    #convert dictionary into pandas dataframe, used pd.Series to assure all parameter lengths are equal
    df = pd.DataFrame([pd.Series(value, name=k) for k, value in data.items()]).T
    
    return df

In [7]:
#initiate beautiful soup html.parser
soup = bs(df['html'][0], 'html.parser')

#utilized find_all to obtain elements location
listings = soup.find_all(class_='articleset listings colset_4')

In [15]:
[x.get_text() for x in listings[0].find_all('li', class_="data-location")]

['Austin, TX\xa078704',
 'Austin, TX\xa078756',
 'Dripping Springs, TX\xa078620',
 'Austin, TX\xa078737',
 'Buda, TX\xa078610',
 'Austin, TX\xa078749',
 'Spicewood, TX\xa078669',
 'Austin, TX\xa078734',
 'Leander, TX\xa078641',
 'Austin, TX\xa078734',
 'Leander, TX\xa078641',
 'Austin, TX\xa078737',
 'Georgetown, TX\xa078626',
 'Austin, TX\xa078717',
 'Austin, TX\xa078729',
 'Round Rock, TX\xa078665',
 'Pflugerville, TX\xa078660',
 'Austin, TX\xa078746',
 'West Lake Hills, TX\xa078746',
 'Austin, TX\xa078731',
 'Austin, TX\xa078759',
 'Austin, TX\xa078737',
 'Liberty Hill, TX\xa078642',
 'Austin, TX\xa078723']

In [20]:
data = df[['html', 'time_scraped']]

dataframe = []
for x in data['html']:
    dt = convert_data(x)
    dt['time'] = data['time_scraped']
    dataframe.append(dt)

In [21]:
total_df = pd.concat(dataframe)

In [24]:
total_df.reset_index(drop=True, inplace=True)

In [25]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17310 entries, 0 to 17309
Data columns (total 13 columns):
mlsid           17310 non-null object
subdiv          17310 non-null object
location        17310 non-null object
bedrooms        17288 non-null object
bathrooms       17302 non-null object
sqft            17304 non-null object
acre            12939 non-null object
data_type       17310 non-null object
price           17310 non-null object
status          17310 non-null object
status_extra    17310 non-null object
photo           17310 non-null object
time            17310 non-null object
dtypes: object(13)
memory usage: 1.7+ MB
